In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import GPy
import itertools
%matplotlib inline
import math
from sklearn.datasets import fetch_mldata
from pandas.tools.plotting import autocorrelation_plot

In [2]:
class data(object):
    #first functional and 1-d structures
    raw_data = None
    struct_data = None
    all_train = None
    all_test = None
    
    #will keep the augmented stuff separate this is functional aug
    aug_raw_data = None
    aug_struct_data = None
    all_aug_train = None
    all_aug_test = None
    samp_aug_train = None
    samp_aug_test = None
    
    #for the ou seasonality
    seasonal = None
    
    def __init__(self, seed, name, years, days, ob_year, ob_day, reset, add_seas, laplace, sigma):
        self.seed = seed
        self.name = name
        self.years = years
        self.days = days
        self.ob_year = ob_year
        self.ob_day = ob_day
        self.reset = reset
        
        #for OU process
        self.mu= 0.0 #still looking at ou cal v GP
        self.theta=0.008
        self.sigma= sigma #1.2#0.1#0.01 #0.4
        
        #fix for the simulations
        self.sim_adjustment_val = 0
        self.add_seas = add_seas
        self.laplace = laplace
        self.get_raw_data()
    
    def get_raw_data(self):
        n = list(range(1,self.years+1))
        ob = list(range(1,self.days+1))
        c = list(itertools.product(n, ob))
        
        flat_combo = [[elem[0],elem[1]] for elem in c]
        self.raw_data = pd.DataFrame(flat_combo, columns=['Year', 'Day'])   
        self.raw_data['Ones']=1
        self.raw_data['Day_Index'] = self.raw_data['Ones'].cumsum()
        self.raw_data.drop('Ones', axis=1, inplace=True)  
    
        self.raw_data['TPrices'] = self.OU(self.mu, self.theta, self.sigma)
        self.raw_data['NoiselessPrice'] = self.raw_data['TPrices']
        self.raw_data['OPrice']= self.raw_data['TPrices'].shift(1)
        self.raw_data['OPrice'][0]= self.raw_data['OPrice'][1]
        
    
    def get_aug_raw_data(self):
        n=list(range(1,self.years+1))
        ob=list(range(1,self.days+1))
        c = list(itertools.product(n, ob))
        tar=list(range(1,self.days+1))
        combo = list(itertools.product(c,tar))
        flat_combo = [[elem[0][0],elem[0][1],elem[1]] for elem in combo]

        self.aug_raw_data = pd.DataFrame(flat_combo, columns=['Year', 'Ob_day', 'Tar_day',])
        self.aug_raw_data['Delta']= self.aug_raw_data['Tar_day'] - self.aug_raw_data['Ob_day']

        prices = self.OU(self.mu, self.theta, self.sigma)
        
        pd_func = pd.DataFrame(prices)
        repeat_stock_num = int(self.aug_raw_data.shape[0]/ pd_func.shape[0])
        
        
        #needs tidying but here I am creating a signal feature for augmented
        df = pd_func.copy()
        df['OPrice'] = pd_func.shift(1)

        df['Up'] = df['OPrice']>df['OPrice'].shift(1)
        df['Plus'] = (df['OPrice']<3.2)&(df['Up']==True)
        df['Minus'] = (df['OPrice']>-3.2)&(df['Up']==False)
        df['Flat'] = (df['Minus']==False) &(df['Plus']==False)
        df['S1']=0
        df['S2']=0
        df['S3']=0

        df.loc[df['Plus']==True,'S1']=1
        df.loc[df['Minus']==True,'S2']=1
        df.loc[df['Flat']==True,'S3']=1

        df.drop(['Up','Plus','Minus','Flat'],axis=1, inplace=True)
        #print(df)
        
        #ob price
        lastprice = list(pd_func.shift(1).values)
        lastprice[0]=lastprice[1] #for now just copy the first price
        lastprice = np.array(lastprice*repeat_stock_num).reshape(self.days,-1).T
        self.aug_raw_data['OPrice'] = lastprice.reshape(-1)
        
        #Signal
        raw_signal = list(df['S1'].values)
        signal = np.array(raw_signal*repeat_stock_num).reshape(self.days,-1).T
        self.aug_raw_data['S1'] = signal.reshape(-1)
        
        raw_signal = list(df['S2'].values)
        signal = np.array(raw_signal*repeat_stock_num).reshape(self.days,-1).T
        self.aug_raw_data['S2'] = signal.reshape(-1)
        
        raw_signal = list(df['S3'].values)
        signal = np.array(raw_signal*repeat_stock_num).reshape(self.days,-1).T
        self.aug_raw_data['S3'] = signal.reshape(-1)
        
        
        #trpice
        self.aug_raw_data['TPrices'] = np.tile(prices.reshape(self.years,self.days),self.days).reshape((-1,))
                                                                                                                     
        #ditch values where we look back in time
        self.aug_raw_data = self.aug_raw_data[self.aug_raw_data["Delta"] >= 0]
        self.aug_raw_data = self.aug_raw_data.reset_index(drop=True)
                
    def mk_tr_tst(self):
        #create test set and train set as well as being able to preprocess
        self.struct_data = self.raw_data.copy()

        #create copies of yes/ ob day and tar day in order to preprocess some and easily be able to refer to days
        self.struct_data['Year_copy']= self.struct_data['Year']
        self.struct_data['Day_copy']= self.struct_data['Day']

        #how much training data 
        train_days = self.ob_year*self.days+self.ob_day
        all_train_data = self.struct_data[:train_days] 

        #fit our scaling on all our training data, we will apply the same to test
        #std_scale = preprocessing.StandardScaler().fit(all_train_data)
        #std_scale_TP = preprocessing.StandardScaler().fit(all_train_data['TPrices'])

        #now transform all our data on the train data scaling keep last two columns so i can still index
        #all_scaled_data = self.struct_data #std_scale.transform(self.struct_data)
        #self.struct_data[self.struct_data.columns[:-2]]=all_scaled_data[:,:-2]

        firstprice_bool = (self.struct_data['Day_copy']==1) 
        firstprices = list(self.struct_data[firstprice_bool]['TPrices'])
        
        #reset prices to zero each year a la chapados if this option is chosen
        if self.reset:
            for i in range(1,self.years+1):
                row_index =  self.struct_data['Year_copy']==i
                self.struct_data.loc[row_index, 'TPrices'] -= firstprices[i-1]

        self.all_train = self.struct_data[:train_days]

        #just test on everything for now 
        bool_test = (self.struct_data['Year_copy']>=0) & (self.struct_data['Year_copy']<self.ob_year+2)
        self.all_test = self.struct_data[bool_test]  

        #for outside experiments I will need to scale the data in the same way
        #return std_scale_TP
    
    def mk_aug_tr_tst(self, max_delta):
        #create test set and train set as well as being able to preprocess
        self.aug_struct_data = self.aug_raw_data.copy()

        #create copies of yes/ ob day and tar day in order to preprocess some and easily be able to refer to days
        self.aug_struct_data['Year_copy']= self.aug_struct_data['Year']
        self.aug_struct_data['Ob_day_copy']= self.aug_struct_data['Ob_day']
        self.aug_struct_data['Tar_day_copy']= self.aug_struct_data['Tar_day']
        self.aug_struct_data['Delta_copy']= self.aug_struct_data['Delta'] 

        #the train and test should overlap over the last data point - take this off when I am sure it is working
        tr_data_bool = (~((self.aug_struct_data['Year']>=self.ob_year+1) | (self.aug_struct_data['Delta']>max_delta))
                           |((self.aug_struct_data['Year']==self.ob_year+1) & (self.aug_struct_data['Delta']<max_delta) 
                            & (self.aug_struct_data['Ob_day']+self.aug_struct_data['Delta']<=self.ob_day)))

        self.all_aug_train = self.aug_struct_data[tr_data_bool]

        #fit our scaling on all our training data, we will apply the same to test
        #std_scale = preprocessing.StandardScaler().fit(self.all_aug_train)
        #std_scale_TP = preprocessing.StandardScaler().fit(self.all_aug_train['TPrices'])

        #transform our training data, but leave the last 4 copy columns which we use for checks and indexing
        #scaled_train_data = self.all_aug_train#std_scale.transform(self.all_aug_train)
        #self.all_aug_train[self.all_aug_train.columns[:-4]]=scaled_train_data[:,:-4]
        
        """
        ###examine scaling and inverse scaling as well as setting each years prices to begin with 0
        firstprice_bool = (all_train_data['Ob_day_copy']==1) & (all_train_data['Delta_copy']==0)
        all_train_data[all_train_data['Year_copy']==1]

        firstprices = list(all_train_data[firstprice_bool]['TPrices'])

        all_train_data_test = all_train_data

        #10 years
        num_years = 10
        for i in range(1,num_years+1):
            row_index =  all_train_data['Year_copy']==i
            all_train_data.loc[row_index, 'TPrices'] -= firstprices[i-1]

        all_train_data.head()
        """
        
        #do the same for the test data again hard coded right now
        tst_data_bool = ((self.aug_struct_data['Year']==self.ob_year+1)  & (self.aug_struct_data['Ob_day']==self.ob_day)
                        & (self.aug_struct_data['Delta']>0))
        self.all_aug_test= self.aug_struct_data[tst_data_bool]
        
        #scaled_test_data = self.all_aug_test#std_scale.transform(self.all_aug_test)
        #self.all_aug_test[self.all_aug_test.columns[:-4]]=scaled_test_data[:,:-4]
        
        #return std_scale_TP
        
    def sample(self,num_samples):
    
        #add on all delta zeros after
        sample_bool = ((self.all_aug_train['Ob_day_copy'])!=(self.all_aug_train['Tar_day_copy']))

        train_data = self.all_aug_train[sample_bool].reset_index(drop=True)

        #sampling function
        rows = np.random.choice(train_data.shape[0], num_samples, replace=False)
        rows = np.sort(rows)

        sampled_train = train_data.ix[rows]
        df_XYtrain = pd.DataFrame(sampled_train) 
        delta_zero = self.all_aug_train[(self.all_aug_train['Delta_copy']==0)]
        #how we sample seems important - go out to max delta...not including all delta zeros
        #but from the beginning of that year
        #print(delta_zero[delta_zero['Year_copy']==10])
        self.samp_aug_train = pd.concat([df_XYtrain, delta_zero[self.ob_year*self.days:]], axis=0)
        #self.samp_aug_train = df_XYtrain
    
        self.samp_aug_test = pd.concat([delta_zero, self.all_aug_test], axis=0)
        
    def plot_auto(self, filename=None):
        autocorrelation_plot(self.struct_data['TPrices'])
        if filename:
            plt.savefig('./figs/'+filename)
            
    def plot(self, filename="stock"):
        if self.name == "ou":
            plt.title("OU process")
            plt.ylabel("Price")
            plt.plot(self.struct_data['TPrices'],'-b')
            plt.savefig('./figs/ou.png')
    
    def plot_curves(self,filename="functional"):
        #Now look at the functional version
        X_train = self.all_train
        for i in range(1,self.years+1):
            prices = X_train[X_train['Year_copy']==i]['TPrices']
            day  = X_train[X_train['Year_copy']==i]['Day_copy']
            ymin, ymax = plt.ylim()
            plt.vlines(self.ob_day,ymin,ymax)
            plt.plot(day, prices)
        #plt.savefig('./figs/'+filename)
     
    def OU(self, mu, theta, sigma):
        #The above discretisation is only valid for very small dt
        # Gillespie 1996, See also “Monte Carlo Simulation of Stochastic Processes”
    
        np.random.seed(self.seed)
        t_0 = 1      # define model parameter
        t_end = self.years*self.days
        length = self.years*self.days

        t = np.linspace(t_0,t_end,length)  # define time axis
        dt = np.mean(np.diff(t))

        y_gill = np.zeros(length)
        y_gill[0] = 0 #use a fixed starting point#np.random.normal(loc=0.0,scale=1.0)

        drift = lambda y,t: theta*(mu-y)      # define drift term
        diffusion = lambda y,t: sigma# define diffusion term
        noise = np.random.normal(loc=0.0,scale=1.0,size=length)*np.sqrt(dt) #define noise process
        
        #laplace = True
        #fat tails laplace noise
        if laplace:
            noise = np.random.laplace(loc=0.0,scale=1.0,size=length)*np.sqrt(dt)

        # solve SDE
        for i in range(1,length):
            y_gill[i] = np.exp(-theta*dt)*y_gill[i-1] + (1- np.exp(-theta*dt))*mu  \
                                              + sigma*noise[i]* np.sqrt((1-np.exp(-2*theta*dt)/2*theta))
        #add more structure
        #note one subtlety if i am adding ou noise i currently want it unrelated to the function i add to
        #when simulating going forward from a certain level we simulate from the level of the function and
        #the noise together - so simulate taking off the level of the function at that point and then add
        #that noise to the function - unfortunately this hadn't been designed from the start and the value
        #we need to take off is hidden in here...lets call it sim_adjustment_val
        
        #add_seas = True
        if add_seas:
            
            #seasonal = np.tile(np.sin(np.linspace(1,250,250)*2*math.pi/250)*4,10)
            seasonal = np.tile(np.sin(np.linspace(1,length/10,length/10)*2*math.pi/(length/10))*4,10)
            self.seasonal = seasonal #-np.mean(seasonal)
            
            #adjustment value so the OU mean reverts according to its vol level, not seasonal too
            self.sim_adjustment_val = self.seasonal[self.days*self.ob_year+self.ob_day]
            return(y_gill+seasonal)
        else:
            return(y_gill)
    
    

In [3]:
class opt(object):
    X=None
    y=None
    Xpred=None
    ytest=None
    ll=None
    var=None
    mod=None
    
    def __init__(self, train_data, test_data, feat_lst, years, days, obs_year, obs_day):
        self.train_data = train_data
        self.test_data = test_data
        self.feat_lst = feat_lst
        self.years = years
        self.days = days
        self.obs_year = obs_year
        self.obs_day = obs_day
        self.prep4opt()
    
    def prep4opt(self):
        self.X = (self.train_data[self.feat_lst].as_matrix())
        self.y = ((self.train_data[['TPrices']]).as_matrix())

        self.Xpred = (self.test_data[self.feat_lst].as_matrix())
        self.ytest = ((self.test_data[['TPrices']]).as_matrix())

    
    def run_opt(self,kern,restarts, aug=False):    
        k_input_d = self.X.shape[1]
        
        if not aug:
            if kern == "rat":
                k = GPy.kern.RatQuad(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True) + GPy.kern.Bias(input_dim=k_input_d)
            elif kern == "mat":
                k = GPy.kern.Matern32(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True) + GPy.kern.Bias(input_dim=k_input_d)
            elif kern == "ou":
                    k = GPy.kern.OU(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True) + GPy.kern.Bias(input_dim=k_input_d)
            elif kern == "rbf":
                k = GPy.kern.RBF(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True) + GPy.kern.Bias(input_dim=k_input_d)
            elif kern == "linear":
                k = GPy.kern.Linear(input_dim=k_input_d,ARD=True)
            elif kern =="per":
                if k_input_d ==1:
                    k=GPy.kern.PeriodicExponential(input_dim=1, variance=1., lengthscale=1.) +\
                    GPy.kern.OU(input_dim=k_input_d, variance=1.0, lengthscale=1.)
                else:
                    k=GPy.kern.PeriodicExponential(input_dim=1, variance=1., lengthscale=1.,active_dims=[1]) +\
                    GPy.kern.OU(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True)
        elif aug:
            if kern == "ou":
                k = GPy.kern.OU(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True) + GPy.kern.Bias(input_dim=k_input_d)    
            #if using feature  
                if k_input_d ==6:
                    k1 = GPy.kern.OU(input_dim=k_input_d-3, variance=1.0, lengthscale=1.,ARD=True, active_dims=[0,1,2]) + GPy.kern.Bias(input_dim=k_input_d) 
                    k2 = GPy.kern.RBF(input_dim=3, variance=1.0, lengthscale=1.,ARD=True, active_dims=[3,4,5])
                    k=k1+k2
                elif k_input_d ==7:
                    k1 = GPy.kern.OU(input_dim=4, variance=1.0, lengthscale=1.,ARD=True, active_dims=[0,1,2,3]) + GPy.kern.Bias(input_dim=k_input_d) 
                    k2 = GPy.kern.RBF(input_dim=3, variance=1.0, lengthscale=1.,ARD=True, active_dims=[4,5,6])
                    k=k1+k2
            elif kern == "rbf":
                if k_input_d ==6:
                    k1 = GPy.kern.RBF(input_dim=k_input_d-3, variance=1.0, lengthscale=1.,ARD=True, active_dims=[0,1,2]) + GPy.kern.Bias(input_dim=k_input_d) 
                    k2 = GPy.kern.RBF(input_dim=3, variance=1.0, lengthscale=1.,ARD=True, active_dims=[3,4,5])
                    k=k1+k2
                elif k_input_d ==7:
                    k1 = GPy.kern.RBF(input_dim=4, variance=1.0, lengthscale=1.,ARD=True, active_dims=[0,1,2,3]) + GPy.kern.Bias(input_dim=k_input_d) 
                    k2 = GPy.kern.RBF(input_dim=3, variance=1.0, lengthscale=1.,ARD=True, active_dims=[4,5,6])
                    k=k1+k2
                k = GPy.kern.Linear(input_dim=k_input_d,ARD=True)
            elif kern == "per":   
                k=GPy.kern.PeriodicExponential(input_dim=1, variance=1., lengthscale=1.,active_dims=[4]) +\
                    GPy.kern.OU(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True)
                k = GPy.kern.Matern32(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True) + GPy.kern.Bias(input_dim=k_input_d)
            elif kern == "rat":
                k = GPy.kern.RatQuad(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True) + GPy.kern.Bias(input_dim=k_input_d)    
            elif kern == "mat":
                k = GPy.kern.Matern32(input_dim=k_input_d, variance=1.0, lengthscale=1.,ARD=True) + GPy.kern.Bias(input_dim=k_input_d)
        
        self.mod = GPy.models.GPRegression(self.X, self.y, k) 
        
        self.mod.constrain_positive('')
        self.mod.optimize(messages=True)
        self.mod.optimize_restarts(num_restarts=restarts)  
        
        #self.mod.Gaussian_noise.constrain_fixed(0.0)
        self.ll = self.mod.log_likelihood()
        #self.ypred, self.var = self.mod.predict_noiseless(self.Xpred)
        self.ypred, self.var = self.mod.predict(self.Xpred)
        return self.ypred, self.var, self.ll


In [4]:
def plot_result1(all_args):
    
    f, axarr= plt.subplots(2,2,figsize=(20, 10))

    for i, arg in enumerate(all_args,1):
        title = arg[0]
        filename = arg[1]
        obs_day = arg[2]
        first_day = arg[3]
        num_days = arg[4]
        xfit = arg[5]
        ypred = arg[6]
        ytest = arg[7]
        var = arg[8]
        ll = arg[9]
             
        if i == 1:
            ax = axarr[0,0]
        elif i==2:
            ax = axarr[0,1]
        elif i==3:
            ax = axarr[1,0]
        elif i==4:
            ax = axarr[1,1]

        print("LL:"+str(ll))
        dypred = 1.96*np.sqrt(var).flatten()
        ypred = ypred.flatten()
        ytest = ytest.flatten()

        ax.plot(xfit[first_day:num_days],ytest[first_day:num_days], 'g-',label='Train Prices')
        ax.plot(xfit[obs_day:num_days],ytest[obs_day:num_days],'b-', label='Test Prices')
        ax.plot(xfit[first_day:num_days],ypred[first_day:num_days], '-', color='black', label='Prediction')
        ax.fill_between(xfit[first_day:obs_day], (ypred[first_day:obs_day]- dypred[first_day:obs_day]), 
                         (ypred[first_day:obs_day] + dypred[first_day:obs_day]), color='blue', alpha=.2)
        ax.fill_between(xfit[obs_day:num_days], (ypred[obs_day:num_days]- dypred[obs_day:num_days]), 
                         (ypred[obs_day:num_days] + dypred[obs_day:num_days]), color='yellow', alpha=.5, label='Conf Int')

        ymin, ymax = ax.get_ylim()
        ax.vlines(obs_day,ymin,ymax)
        #print(title)
        
        ax.set_title(title)
        ax.legend(loc='best')
        ax.grid(which='major')
    #plt.savefig('./figs/'+filename)
    plt.show()

In [5]:
def ou_cal(data,delta,LS=True):
    #Calibrating the OU process - using either maximum likelihood or least squares
    #delta =1.0
    #test_func = y_gill[:pred_day]
    test_func = data
    n = test_func.shape[0]-1

    Sx  = np.sum(test_func[:-1])
    Sy  = np.sum(test_func[1:])
    Sxx = np.sum(test_func[:-1]**2)
    Sxy = np.sum(test_func[:-1]*test_func[1:])
    Syy = np.sum(test_func[1:]**2)

    if LS:
        a  = (n*Sxy - Sx*Sy) / (n*Sxx -Sx**2)
        b  = (Sy - a*Sx)/n
        sd = np.sqrt((n*Syy - Sy**2 - a*(n*Sxy - Sx*Sy))/n/(n-2))

        lbda = -np.log(a)/delta
        mu     = b/(1-a)
        sigma  =  sd * np.sqrt( -2*np.log(a)/delta/(1-a**2))
    else:
        mu  = (Sy*Sxx - Sx*Sxy) / (n*(Sxx - Sxy) - (Sx**2 - Sx*Sy))
        lbda = -np.log( (Sxy - mu*Sx - mu*Sy + n*mu**2) / (Sxx -2*mu*Sx + n*mu**2) ) / delta
        a = np.exp(-lbda*delta)
        sigmah2 = (Syy - 2*a*Sxy + a**2*Sxx - 2*mu*(1-a)*(Sy - a*Sx) + n*mu**2*(1-a)**2)/n
        sigma = np.sqrt(sigmah2*2*lbda/(1-a**2))
        
    return mu,sigma,lbda
#mu, sigma, lbda = ou_cal(y_ou_sin[:pred_day],1.0,True)

In [6]:
def run_sim(opt_data,opt_funcdata,opt_augdata, days, obs_year, obs_day, pred_day, ypred, ypred_func, ypred_aug,
           var, var_func, var_aug, dataset, add_seas, laplace):
    #set params
    #add_seas = True
    #laplace = True
    #start point
    t_0 = 0
    #end point
    t_end = days*(obs_year)+obs_day+50
    length = t_end
    #observ point
    observ_point = pred_day
    #price series
    price_series = opt_data.ytest
    func_price_series = opt_funcdata.ytest
    aug_price_series = opt_augdata.ytest

    #prediction
    onedim_pred = ypred.flatten()
    func_pred = ypred_func.flatten()
    aug_pred = ypred_aug.flatten()
    
    # error bars
    onedim_sd = np.sqrt(var).flatten()
    funcdim_sd = np.sqrt(var_func).flatten()
    augdim_sd = np.sqrt(var_aug).flatten()
    
    #underlying stoch process params
    mu = dataset.mu
    theta = dataset.theta
    sigma = dataset.sigma

    #########################
    t = np.linspace(t_0,t_end,length)
    dt = np.mean(np.diff(t))
    drift = lambda y,t: theta*(mu-y)      # define drift term
    diffusion = lambda y,t: sigma# define diffusion term

    #noise process
    noise = np.random.normal(loc=0.0,scale=1.0,size=length)*np.sqrt(dt) #define noise process

    # solve SDE
    #Ahrash suggestion to have the same starting point for prediction

    mu_cal, sigma_cal, theta_cal = ou_cal(opt_data.y,1.0,False)    

    t_0 = pred_day
    length = t_end-t_0

    num_sims = 1000
    y_all = np.zeros([length,num_sims])
    y_all_cal = np.zeros([length,num_sims])
    y_all_func = np.zeros([length,num_sims])
    y_all_cal_func = np.zeros([length,num_sims])
    y_all_aug = np.zeros([length,num_sims])
    y_all_cal_aug = np.zeros([length,num_sims])
    
    for j in range(num_sims):

        drift = lambda y,t1: theta*(mu-y)      # define drift term
        diffusion = lambda y,t1: sigma# define diffusion term
        noise = np.random.normal(loc=0.0,scale=1.0,size=length)*np.sqrt(dt) #define noise process
        noise1 = np.random.normal(loc=0.0,scale=1.0,size=length)*np.sqrt(dt)

        #make the noise mre difficult laplace - real returns are fat tailed
        if laplace:
            noise = np.random.laplace(loc=0.0,scale=1.0,size=length)*np.sqrt(dt)
            noise1 = np.random.laplace(loc=0.0,scale=1.0,size=length)*np.sqrt(dt)

        y_new = np.zeros(length)
        y_new_cal = np.zeros(length)

        y_new_func = np.zeros(length)
        y_new_cal_func = np.zeros(length)

        y_new_aug = np.zeros(length)
        y_new_cal_aug = np.zeros(length)
    
        #note if we are adding a function we need to take that off else the OU will mean revert too much
        function_adjustment = dataset.sim_adjustment_val
        
        y_new[0] = onedim_pred[pred_day] -function_adjustment # from ahrash note
        y_new_cal[0] = onedim_pred[pred_day] -function_adjustment

        y_new_func[0] = func_pred[pred_day] -function_adjustment # from ahrash note
        y_new_cal_func[0] = func_pred[pred_day] -function_adjustment

        y_new_aug[0] = aug_pred[pred_day] -function_adjustment # from ahrash note
        y_new_cal_aug[0] = aug_pred[pred_day] -function_adjustment

        t1 = np.linspace(t_0,t_end,length)  # define time axis
        dt = np.mean(np.diff(t1))

        # solve SDE note i am adding laplace noise to the cal one - which is helping it too much
        # although this could be an interesting aside

        for i in range(1,length):

            y_new[i] = np.exp(-theta*dt)*y_new[i-1] + (1- np.exp(-theta*dt))*mu  \
                                              + sigma*noise[i]* np.sqrt((1-np.exp(-2*theta*dt)/2*theta))

            y_new_cal[i] = np.exp(-theta_cal*dt)*y_new_cal[i-1] + (1- np.exp(-theta_cal*dt))*mu_cal  \
                                              + sigma_cal*noise1[i]* np.sqrt((1-np.exp(-2*theta_cal*dt)/2*theta_cal))

            y_new_func[i] = np.exp(-theta*dt)*y_new_func[i-1] + (1- np.exp(-theta*dt))*mu  \
                                              + sigma*noise[i]* np.sqrt((1-np.exp(-2*theta*dt)/2*theta))

            y_new_cal_func[i] = np.exp(-theta_cal*dt)*y_new_cal_func[i-1] + (1- np.exp(-theta_cal*dt))*mu_cal  \
                                              + sigma_cal*noise1[i]* np.sqrt((1-np.exp(-2*theta_cal*dt)/2*theta_cal))

            y_new_aug[i] = np.exp(-theta*dt)*y_new_aug[i-1] + (1- np.exp(-theta*dt))*mu  \
                                              + sigma*noise[i]* np.sqrt((1-np.exp(-2*theta*dt)/2*theta))

            y_new_cal_aug[i] = np.exp(-theta_cal*dt)*y_new_cal_aug[i-1] + (1- np.exp(-theta_cal*dt))*mu_cal  \
                                              + sigma_cal*noise1[i]* np.sqrt((1-np.exp(-2*theta_cal*dt)/2*theta_cal))
            
        y_all[:,j] = y_new 
        y_all_cal[:,j] = y_new_cal

        y_all_func[:,j] = y_new_func
        y_all_cal_func[:,j] = y_new_cal_func

        y_all_aug[:,j] = y_new_aug
        y_all_cal_aug[:,j] = y_new_cal_aug

    #now with seasonal adjustments added in   
    #when we also have a function as well as GP we need to adjust this too
    #add on the OU function adjustment to match with above     

    seasonal = dataset.seasonal 
    ymean = y_all.mean(axis=1)
    ystd = y_all.std(axis=1)
    if add_seas:
        ymean = ymean + seasonal[t_0:t_end]#-seasonal[t_0]

    ymean_func = y_all_func.mean(axis=1)
    ystd_func = y_all_func.std(axis=1)
    if add_seas:
        ymean_func = ymean_func + seasonal[t_0:t_end]#-seasonal[t_0]

    ymean_aug = y_all_aug.mean(axis=1)
    ystd_aug = y_all_aug.std(axis=1)
    if add_seas:
        ymean_aug = ymean_aug + seasonal[t_0:t_end]#-seasonal[t_0]

    ymean_cal = y_all_cal.mean(axis=1)
    ystd_cal = y_all_cal.std(axis=1)
    ymean_cal = ymean_cal #note the AR(1) doesnt know about the function so adjustment the
    #the other way here!
    if add_seas:
        ymean_cal = ymean_cal +function_adjustment

    ymean_cal_func = y_all_cal_func.mean(axis=1)
    ystd_cal_func = y_all_cal_func.std(axis=1)
    ymean_cal_func = ymean_cal_func 

    ymean_cal_aug = y_all_cal_aug.mean(axis=1)
    ystd_cal_aug = y_all_cal_aug.std(axis=1)
    ymean_cal_aug = ymean_cal_aug 
    
    ret_vars = (t1,ymean, ystd, ymean_cal, ystd_cal, onedim_pred, t_0, t_end, onedim_sd, func_pred, ymean_func,
               ystd_func, funcdim_sd, aug_pred, ymean_aug, ystd_aug, augdim_sd,mu_cal, sigma_cal, theta_cal)
    return ret_vars

In [7]:
def mk_tst_metric(ymean,ymean_cal,onedim_pred, func_pred, aug_pred, ystd, ystd_cal, onedim_sd, ystd_func, funcdim_sd
                 ,ystd_aug, augdim_sd):
    
    mse_cal = np.zeros(ymean.shape[0])
    mse_1d = np.zeros(ymean.shape[0])
    mse_func = np.zeros(ymean.shape[0])
    mse_aug = np.zeros(ymean.shape[0])

    for i in range(ymean.shape[0]):
        mse_cal[i] = np.cumsum((ymean-ymean_cal)**2)[i]/(i+1)
        mse_1d[i] = np.cumsum((ymean-onedim_pred[t_0:t_end])**2)[i]/(i+1)
        mse_func[i] = np.cumsum((ymean_func-func_pred[t_0:t_end])**2)[i]/(i+1)
        mse_aug[i] = np.cumsum((ymean_aug-aug_pred[t_0:t_end])**2)[i]/(i+1)

    mse_1d_std = np.zeros(ymean.shape[0])
    mse_func_std = np.zeros(ymean.shape[0])
    mse_aug_std = np.zeros(ymean.shape[0])
    mse_cal_std = np.zeros(ymean.shape[0])
    
    for i in range(ymean.shape[0]):
        mse_cal_std[i] = np.cumsum((ystd-ystd_cal)**2)[i]/(i+1)
        mse_1d_std[i] = np.cumsum((ystd-onedim_sd[t_0:t_end])**2)[i]/(i+1)
        mse_func_std[i] = np.cumsum((ystd_func-funcdim_sd[t_0:t_end])**2)[i]/(i+1)
        mse_aug_std[i] = np.cumsum((ystd_aug-augdim_sd[t_0:t_end])**2)[i]/(i+1)
        
    ret_vars = mse_cal, mse_1d, mse_func, mse_aug, mse_1d_std, mse_func_std, mse_aug_std, mse_cal_std
    return ret_vars


In [8]:
def show_preds(ymean,ystd,t1,ymean_cal,ystd_cal,onedim_pred, onedim_sd, func_pred, ymean_func, ystd_func,
              funcdim_sd, aug_pred, ystd_aug, ymean_aug, augdim_sd):

    #f, axarr = plt.subplots(2, 2)
    f, axarr= plt.subplots(2,2,figsize=(20, 10))

    axarr[0, 0].fill_between(t1, (ymean- ystd), 
                     (ymean+ ystd), color='blue', alpha=.3, label='SD')
    axarr[0, 0].plot(t1,ymean_cal, label='cal')
    axarr[0, 0].fill_between(t1, (ymean_cal- ystd_cal), 
                     (ymean_cal+ ystd_cal), color='yellow', alpha=.3, label='Cal SD')
    axarr[0, 0].plot(t1,ymean, label='mean')
    axarr[0, 0].set_title('AR(1)') 

    
    axarr[0, 1].plot(t1,onedim_pred[t_0:t_end], label='1d_pred')
    axarr[0, 1].fill_between(t1, (ymean- ystd), 
                     (ymean+ ystd), color='blue', alpha=.3, label='SD')
    axarr[0, 1].plot(t1,ymean, label='mean')
    axarr[0, 1].fill_between(t1, onedim_pred[t_0:t_end]- onedim_sd[t_0:t_end], 
            (onedim_pred[t_0:t_end] + onedim_sd[t_0:t_end]), color='yellow', alpha=.5, label='Conf Int')
    axarr[0, 1].set_title('1 dim GP')
    #axarr[0, 1].legend() 

    axarr[1, 0].plot(t1,func_pred[t_0:t_end], label='func_pred')
    axarr[1, 0].fill_between(t1, (ymean_func- ystd_func), 
                     (ymean_func+ ystd_func), color='blue', alpha=.3, label='SD')
    axarr[1, 0].plot(t1,ymean_func, label='mean')
    axarr[1, 0].fill_between(t1, func_pred[t_0:t_end]- funcdim_sd[t_0:t_end], 
            (func_pred[t_0:t_end] + funcdim_sd[t_0:t_end]), color='yellow', alpha=.5, label='Conf Int')
    #axarr[1, 0].legend() 
    axarr[1,0].set_title('Func GP')

    axarr[1, 1].plot(t1,aug_pred[t_0:t_end], label='aug_pred')
    axarr[1, 1].fill_between(t1, (ymean_aug- ystd_aug), 
                     (ymean_aug+ ystd_aug), color='blue', alpha=.3, label='SD')
    axarr[1, 1].plot(t1,ymean_aug, label='mean')
    axarr[1, 1].fill_between(t1, aug_pred[t_0:t_end]- augdim_sd[t_0:t_end], 
            (aug_pred[t_0:t_end] + augdim_sd[t_0:t_end]), color='yellow', alpha=.5, label='Conf Int')
    axarr[1,1].set_title('Aug GP')

    
    f.subplots_adjust(hspace=0.3)
    f.subplots_adjust(wspace=0.3)

    #axarr[1, 1].legend() 

In [9]:
def show_mse(mse_cal, mse_1d, mse_func, mse_aug):

    f, axarr = plt.subplots(1,2, sharex=True, figsize=(20, 10))

    axarr[0].plot(mse_cal, label='cal')
    axarr[0].plot(mse_1d, label='1d')
    axarr[0].plot(mse_func, label='func')
    axarr[0].plot(mse_aug, label='aug')

    axarr[0].legend()
    axarr[1].plot(mse_cal_std, label='cal')
    axarr[1].plot(mse_1d_std, label='1d')
    axarr[1].plot(mse_func_std, label='func')
    axarr[1].plot(mse_aug_std, label='aug')
    axarr[1].legend()

In [10]:
class settings(object):
    def __init__(self, years, days, obs_year, obs_day, reset2zero, max_delta, samples, add_seas, laplace, sigma, kern):
        self.years = years
        self.days = days
        self.obs_year = obs_year
        self.obs_day = obs_day
        self.reset2zero = reset2zero
        self.pred_day = days*obs_year+obs_day
        self.max_delta = max_delta
        self.samples = samples
        self.add_seas = add_seas
        self.laplace = laplace
        self.sigma = sigma
        self.kern = kern
        
        self.aug1dyears = 1
        self.aug1ddays = years*days
        self.aug1dobs_year = 0
        self.aug1dobs_day = days*obs_year+obs_day
        self.aug1dpred_day = days*obs_year+obs_day
        

In [11]:
seed = 0
np.random.seed(seed)
add_seas = True
laplace = False
sigma = 0.1 #OU sigma - rest is mean zero, 
kern = "rat"
#Set what data we are using set train/ test periods

years = 10
days = 250
obs_year = 5#0#4#3 #test1
obs_day = 20#800#50 #95
reset2zero =False
#pred_day = days*obs_year+obs_day

#for augmented data set our max look forward and samples - it also has this year's delta zeros
max_delta = 50
samples = 500

s = settings(years,days,obs_year,obs_day,reset2zero,max_delta,samples, add_seas, laplace, sigma, kern)

#optimisation runs
restarts =1

In [12]:
#Create raw datasets

#1d or functional
dataset = data(seed, "ou",s.years,s.days,s.obs_year,s.obs_day, s.reset2zero, s.add_seas, s.laplace, s.sigma)
dataset.mk_tr_tst()
all_train_data, all_test_data = dataset.all_train, dataset.all_test

#augmented
dataset.get_aug_raw_data()
dataset.mk_aug_tr_tst(s.max_delta)
dataset.sample(s.samples)

samp_train_data, samp_test_data = dataset.samp_aug_train, dataset.samp_aug_test

#augmented 1d
dataset1d = data(seed, "ou",s.aug1dyears,s.aug1ddays,s.aug1dobs_year,s.aug1dobs_day, s.reset2zero
                 ,s.add_seas, s.laplace, s.sigma)
dataset1d.get_aug_raw_data()
dataset1d.mk_aug_tr_tst(s.max_delta)
dataset1d.sample(s.samples)

samp_train_data1d, samp_test_data1d = dataset1d.samp_aug_train, dataset1d.samp_aug_test

 /Users/johngoodacre/anaconda/envs/gp/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
#dataset.plot()
#dataset.plot_auto()

In [14]:
#Optimisations
#Basic 1-d GP
feat_lst = ['Day_Index']
opt_data=opt(all_train_data,all_test_data,feat_lst, s.years, s.days, s.obs_year, s.obs_day)

ypred, var, ll = opt_data.run_opt(s.kern,restarts)

#functional 
feat_lst = ['Year', 'Day'] 
opt_funcdata=opt(all_train_data,all_test_data,feat_lst, s.years, s.days, s.obs_year, s.obs_day)

ypred_func, var_func, ll_func = opt_funcdata.run_opt(s.kern,restarts) 

#Augmented
feat_lst = ['Year', 'Ob_day','Delta','OPrice','S1','S2','S3'] 
opt_augdata=opt(samp_train_data,samp_test_data,feat_lst, s.years, s.days, s.obs_year, s.obs_day)

ypred_aug, var_aug, ll_aug = opt_augdata.run_opt(s.kern,restarts, True)

#Augmented 1d
feat_lst = ['Ob_day','Delta','OPrice','S1','S2','S3'] 
opt_1daugdata=opt(samp_train_data1d,samp_test_data1d,feat_lst, s.aug1dyears, s.aug1ddays, 
                  s.aug1dobs_year, s.aug1dobs_day)

ypred_1daug, var_1daug, ll_1daug = opt_1daugdata.run_opt(s.kern,restarts, True)

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    03s54  0007   3.498595e+05   1.061813e+11 
    08s76  0017  -9.047836e+02   3.780825e+02 
    14s82  0029  -9.147857e+02   2.192395e+01 
    22s73  0044  -9.161648e+02   4.865254e-01 
    29s13  0056  -9.161930e+02   4.598925e-07 
    29s73  0057  -9.161930e+02   4.598925e-07 
Runtime:     29s73
Optimization status: Converged

Optimization restart 1/1, f = -916.1930449807369
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    01s18  0002   1.988838e+03   2.211788e+05 
    06s36  0012  -9.427588e+02   1.116912e+02 
    12s32  0024  -9.588009e+02   2.711394e+01 
    24s20  0047  -9.604895e+02   6.044376e-01 
    26s44  0051  -9.604906e+02   2.375465e-04 
    27s38  0053  -9.604906e+02   2.484954e-04 
Runtime:     27s38
Optimization status: Converged

Optimization restart 1/1, f = -960.4905895486545
Running L-BFGS-B (Scipy implementation) Code:
  runt

 /Users/johngoodacre/anaconda/envs/gp/lib/python3.6/site-packages/paramz-0.7.3-py3.6.egg/paramz/transformations.py:109: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/1, f = -12.307705588962676
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s00  0000   5.149627e+03           nan 
    09s25  0009  -2.600591e+02   2.958738e+05 
    18s58  0018  -8.949662e+02   2.315427e+03 
    33s49  0031  -9.462222e+02   6.813503e+00 
    58s17  0056  -9.659079e+02   2.961778e-01 
 01m01s46  0060  -9.659078e+02   7.949359e-01 
 01m03s17  0062  -9.659096e+02   7.394329e-05 
Runtime:  01m03s17
Optimization status: Converged

Optimization restart 1/1, f = -965.9096301965885


In [15]:
#Produce charts

title = "GP regression optimised"
filename = "opt.png"
first_day = s.days*s.obs_year+1
num_days = s.days*(s.obs_year+1)
obs_day = s.days*s.obs_year+s.obs_day

xfit = np.linspace(0, len(ypred), len(ypred)).reshape(len(ypred), 1).flatten()
ytest = opt_data.ytest
onedargs = (title, filename, obs_day, first_day, num_days, xfit, ypred, ytest, var, ll)

title = "func GP regression optimised"
filename = "funcopt.png"
ytest_func = opt_funcdata.ytest
funcargs = (title, filename, obs_day, first_day, num_days, xfit, ypred_func, ytest_func, var_func, ll_func)

title = "Aug GP regression optimised"
filename = "augopt.png"
ytest_aug = opt_augdata.ytest
augargs = (title, filename, obs_day, first_day, num_days, xfit, ypred_aug, ytest_aug, var_aug, ll_aug)


title = "Aug GP 1d regression optimised"
filename = "aug1dopt.png"
ytest_aug1d = opt_1daugdata.ytest
aug1dargs = (title, filename, s.aug1dobs_day, first_day, num_days, xfit, ypred_1daug, ytest_aug1d, var_1daug, ll_1daug)

all_args = (onedargs,funcargs,augargs,aug1dargs)
#plot_result1(all_args)

In [16]:
t1, ymean, ystd, ymean_cal, ystd_cal, onedim_pred, t_0, t_end, onedim_sd, func_pred, ymean_func, ystd_func,\
    funcdim_sd, aug_pred, ymean_aug, ystd_aug, augdim_sd, mu_cal, sigma_cal, theta_cal \
                = run_sim(opt_data,opt_funcdata,opt_augdata, s.days, s.obs_year, s.obs_day, s.pred_day, 
                          ypred, ypred_func, ypred_aug, var, var_func, var_aug, dataset, add_seas,laplace)

In [17]:
mse_cal, mse_1d, mse_func, mse_aug, mse_1d_std, mse_func_std, mse_aug_std, mse_cal_std \
    =mk_tst_metric(ymean,ymean_cal,onedim_pred, func_pred, aug_pred, ystd, ystd_cal, onedim_sd, ystd_func, funcdim_sd
                 ,ystd_aug, augdim_sd)

In [18]:
#show_preds(ymean,ystd,t1,ymean_cal,ystd_cal,onedim_pred, onedim_sd, func_pred, ymean_func, ystd_func,
#              funcdim_sd, aug_pred,ystd_aug, ymean_aug, augdim_sd)

In [19]:
#show_mse(mse_cal, mse_1d, mse_func, mse_aug)

In [20]:
#print(mu_cal)
#print(sigma_cal)
#print(theta_cal)

In [21]:
columns = ['Seed','Pred_day','Max_Delta','Samples', 'Kernel', 'Noise', 'Laplace','Pred_level',
           'AR1_MSE_10','AR1_MSE_20','AR1_MSE_30', 'AR1_MSESD_10', 'AR1_MSESD_20', 'AR1_MSESD_30',
          '1D_MSE_10','1D_MSE_20','1D_MSE_30','1D_MSESD_10','1D_MSESD_20','1D_MSESD_30',
          'FUNC_MSE_10','FUNC_MSE_20','FUNC_MSE_30','FUNC_MSESD_10','FUNC_MSESD_20','FUNC_MSESD_30',
          'AUG_MSE_10','AUG_MSE_20','AUG_MSE_30','AUG_MSESD_10','AUG_MSESD_20','AUG_MSESD_30'] #,
          #'AUD1D_MSE_10','AUG1D_MSE_20','AUG1D_MSE_30','AUG1D_MSESD_10','AUG1D_MSE_20','AUG1D_MSE_30']
results = pd.DataFrame(columns=columns)

In [22]:
run_results = [seed,s.pred_day,max_delta, s.samples, s.kern,s.sigma,s.laplace,ymean[0]]
ar1 = [mse_cal[10],mse_cal[20],mse_cal[30],mse_cal_std[10],mse_cal_std[20],mse_cal_std[30]]
oned = [mse_1d[10],mse_1d[20],mse_1d[30],mse_1d_std[10],mse_1d_std[20],mse_1d_std[30]]
func = [mse_func[10],mse_func[20],mse_func[30],mse_func_std[10],mse_func_std[20],mse_func_std[30]]
aug = [mse_aug[10],mse_aug[20],mse_aug[30],mse_aug_std[10],mse_aug_std[20],mse_aug_std[30]]

run_results = run_results+ ar1+oned+func+aug

In [23]:
row = pd.Series(run_results, columns)

In [24]:
results = results.append([row], ignore_index=True)

In [25]:
results

,Seed,Pred_day,Max_Delta,Samples,Kernel,Noise,Laplace,Pred_level,AR1_MSE_10,AR1_MSE_20,...,FUNC_MSE_30,FUNC_MSESD_10,FUNC_MSESD_20,FUNC_MSESD_30,AUG_MSE_10,AUG_MSE_20,AUG_MSE_30,AUG_MSESD_10,AUG_MSESD_20,AUG_MSESD_30
0,0,1270,50,500,rat,0.1,False,2.932185,0.186888,0.607887,...,0.288285,0.042395,0.073805,0.081213,0.009217,0.031246,0.043085,0.003065,0.002007,0.004246
